# Notebook for the analysis and creation of plots and tables for the report

In [29]:
import pandas as pd
import numpy as np

# Load results.csv 

- Tables for delta sarc ratio with confidence intervals and marginals. 
- The same table but just for up and down results.
- Something with the flip rate 
- Plots: 
  - Heatmap input to second round. Let's make it according to claims. 
  - Label distribution: Adjust so that we can see the small percentage. 
  - Valid-output distribution for both rounds. 

### Helper functions

In [30]:
def ci95(x):
    '''
    Helper function to return the 95% Confidence Interval around the mean. 
    '''
    n = x.count()
    if n <= 1:
        return np.nan
    se = x.std(ddof=1) / np.sqrt(n)
    return 1.96 * se

In [31]:
def fmt(m, c):
    if pd.isna(m) or pd.isna(c):
        return "NaN"
    return f"{m:.3f} ± {c:.3f}"

## EDA Sarcasm Ratio

## Delta sarc ratio table

In [315]:
# def make_delta_table(df, filter = None):
#     columns = ['delta_sarc', 'model_receiver', 'model_sender']

#     for c in columns:
#         if c not in df.columns:
#             raise ValueError(f'Missing required column: {c}')
    
#     if filter is None:
#         df_filter = df[columns].copy()
#     elif filter == 'up':
#         df_filter = df[df['flip_direction'] == 'up'].copy()
#         df_filter = df_filter[columns].copy()
#     elif filter == 'down':
#         df_filter = df[df['flip_direction'] == 'down'].copy()
#         df_filter = df_filter[columns].copy()
#     else:
#         raise ValueError('Filter not defined')
    
#     df_filter['abs_delta_sarc'] = df_filter['delta_sarc'].abs()

#     df_filter.drop('delta_sarc', inplace = True, axis = 1)

#     df_sender_receiver = (df_filter.groupby(['model_receiver', 'model_sender'])['abs_delta_sarc'].agg(mean = 'mean', count = 'count', ci95=ci95).reset_index())

#     df_sender = (df_filter.groupby('model_sender')['abs_delta_sarc'].agg(mean = 'mean', count = 'count', ci95=ci95).reset_index())
#     df_receiver = (df_filter.groupby('model_receiver')['abs_delta_sarc'].agg(mean = 'mean', count = 'count', ci95=ci95).reset_index())

#     print(df_receiver)
#     # align marginals with main table
#     sender_stats = df_sender.set_index('model_sender')
#     receiver_stats = df_receiver.set_index('model_receiver')


#     pivot_mean = df_sender_receiver.pivot(
#         index='model_sender',
#         columns = 'model_receiver',
#         values = 'mean'
#     )
    
#     pivot_ci = df_sender_receiver.pivot(
#         index='model_sender',
#         columns='model_receiver',
#         values='ci95'
#     )

#     df_table = pivot_mean.copy().astype(object)
#     for r in df_table.index:
#         for c in df_table.columns:
#             m = pivot_mean.loc[r, c]
#             ci = pivot_ci.loc[r, c]
#             df_table.loc[r, c] = fmt(m, ci)


#     sender_row = []
#     for r in df_table.index:
#         m = sender_stats.loc[r, 'mean']
#         ci = sender_stats.loc[r, 'ci95']
#         sender_row.append(fmt(m, ci))
#     df_table['sender_marg'] = sender_row


#     print(f'Receiver stats index :{receiver_stats.index}')
#     receiver_column = {}
#     for s in df_table.columns[:-1]:
#         print(s)
#         if s in receiver_stats.index:
#             m = receiver_stats.loc[s, 'mean']
#             ci = receiver_stats.loc[s, 'ci95']
#             receiver_column[s] = fmt(m, ci)
#         else:
#             receiver_column[s] = "--"

#     # sender_row['receiver_marg'] = fmt(global_mean, global_ci)

#     df_table.loc['receiver_marg'] = receiver_column
#     df_table.columns = df_table.columns.str.replace('_', '')
#     df_table.index = df_table.index.str.replace('_', '')

#     # ---- LaTeX table (now with mean ± ci95 strings) ----
#     table = df_table.to_latex(
#         index=True,
#         caption="Abs delta sarcasm by sender/receiver",
#         label="tab:abs_delta_sarc",
#         bold_rows=False,
#         na_rep="--",
#         escape=False
#     )

#     return df_table, table
    

In [115]:
results = pd.read_csv('results/results.csv')

In [116]:
print(results.head())

   id model_receiver    model_sender  valid_json_count_r2  round2_sarc_ratio  \
0   1   llama-3.2-3b  mistral-0.2-7b                   10                1.0   
1   3   llama-3.2-3b  mistral-0.2-7b                   10                0.8   
2  32   llama-3.2-3b  mistral-0.2-7b                   10                1.0   
3  33   llama-3.2-3b  mistral-0.2-7b                   10                0.5   
4  38   llama-3.2-3b  mistral-0.2-7b                   10                0.3   

   round1_sarc_ratio  delta_sarc flip_direction  flip  
0                1.0         0.0           none     0  
1                1.0        -0.2           none     0  
2                1.0         0.0           none     0  
3                0.9        -0.4           none     0  
4                0.9        -0.6           down     1  


In [450]:
def make_delta_table(df, filter = None):
    columns = ['delta_sarc', 'model_receiver', 'model_sender']

    for c in columns:
        if c not in df.columns:
            raise ValueError(f'Missing required column: {c}')
    
    if filter is None:
        df_filter = df[columns].copy()
    elif filter == 'up':
        df_filter = df[df['round1_sarc_ratio'] < 0.5].copy()
        df_filter = df_filter[columns].copy()
    elif filter == 'down':
        df_filter = df[df['round1_sarc_ratio'] >= 0.5].copy()
        df_filter = df_filter[columns].copy()
    else:
        raise ValueError('Filter not defined')
    
    df_filter['abs_delta_sarc'] = df_filter['delta_sarc'].abs()
    df_filter.drop('delta_sarc', inplace = True, axis = 1)

    df_sender_receiver = (df_filter.groupby(['model_receiver', 'model_sender'])['abs_delta_sarc'].agg(mean = 'mean', count = 'count', ci95=ci95).reset_index())

    
    pivot_mean = df_sender_receiver.pivot(
        index='model_sender',
        columns = 'model_receiver',
        values = 'mean'
    ).round(2)
    
    pivot_ci = df_sender_receiver.pivot(
        index='model_sender',
        columns='model_receiver',
        values='ci95',
    ).round(3)

    # Add a row label column
    mean_labeled = pivot_mean.copy()
    mean_labeled.index = mean_labeled.index + "_A_mean"

    ci_labeled = pivot_ci.copy()
    ci_labeled.index = ci_labeled.index + "_ci95"

    combined = (
        pd.concat([mean_labeled, ci_labeled], axis=0).sort_index(kind='merge')
    )

    macro_avg_sender = mean_labeled.mean(axis=1, skipna=True)
    macro_avg_receiver = mean_labeled.mean(axis=0, skipna=True)


    combined['avg'] = macro_avg_sender
    combined.loc['avg'] = macro_avg_receiver

    return combined

In [451]:
def make_latex_body(combined, float_fmt="{:.6f}"):
    df = combined.copy()

    base_models = []
    for idx in df.index:
        if idx.endswith("_A_mean"):
            base = idx[:-len("_A_mean")]
        elif idx.endswith("_ci95"):
            base = idx[:-len("_ci95")]
        else:
            continue
        if base not in base_models:
            base_models.append(base)

    def format_row(label, values):
        cells = []
        for v in values:
            if pd.isna(v):
                cells.append("--")
            else:
                cells.append(float_fmt.format(v))
        return "\\texttt{" + label + '}' + "&" + " & ".join(cells) + r" \\"
    
    def format_row_ci(label, values):
        cells = []
        for v in values[:-1]:
            if pd.isna(v):
                cells.append("")
            elif label in ('marg', 'avg'):
                cells.append("")
            else:
                v = '('+ u"\u00B1" + float_fmt.format(v) + ')'
                v = "\\footnotesize{" + v + "}"
                cells.append(v)
        cells.append('')

        row = f" & " + " & ".join(cells) + r" \\"
        return row

    header = list(base_models) + ['Avg.']
    header_tex = ["\\texttt{" + h + "}" for h in header]
    header_str = " & " + " & ".join(header_tex) + r" \\" + '\n' + '\\midrule'
    lines = [header_str]
    base_models.append('avg')

    for base in base_models:
        if base != 'avg':
            ci_idx = base + "_ci95"
            mean_idx = base + "_A_mean"
            ci_vals = df.loc[ci_idx].values
            mean_vals = df.loc[mean_idx].values
        else:
            lines.append('\\midrule')
            mean_idx = 'avg'
            mean_vals = df.loc[mean_idx].values
            ci_vals = df.loc[ci_idx].values
    

        lines.append(format_row(base, mean_vals))
        lines.append(format_row_ci(base, ci_vals))
    lines.pop()

    return "\n".join(lines)


In [ ]:
results[results['round1_sarc_ratio'] >= 0.5]

,id,model_receiver,model_sender,valid_json_count_r2,round2_sarc_ratio,round1_sarc_ratio,delta_sarc,flip_direction,flip
0,1,llama-3.2-3b,mistral-0.2-7b,10,1.0,1.0,0.0,none,0
1,3,llama-3.2-3b,mistral-0.2-7b,10,0.8,1.0,-0.2,none,0
2,32,llama-3.2-3b,mistral-0.2-7b,10,1.0,1.0,0.0,none,0
3,33,llama-3.2-3b,mistral-0.2-7b,10,0.5,0.9,-0.4,none,0
4,38,llama-3.2-3b,mistral-0.2-7b,10,0.3,0.9,-0.6,down,1
...,...,...,...,...,...,...,...,...,...
711260,205175,mistral-0.2-7b,qwen-2.5-7b,10,0.0,1.0,-1.0,down,1
711270,205330,mistral-0.2-7b,qwen-2.5-7b,10,1.0,1.0,0.0,none,0
711271,205332,mistral-0.2-7b,qwen-2.5-7b,10,0.0,1.0,-1.0,down,1
711276,205381,mistral-0.2-7b,qwen-2.5-7b,10,1.0,1.0,0.0,none,0


In [462]:
def check(df):
    df = df.copy()
    df['abs'] = df['delta_sarc'].abs()

    df_agg = df.groupby(['model_sender', 'model_receiver'])['abs'].mean()

    print(df_agg)

In [461]:
check(results[results['flip_direction'] == 'down'])

model_sender    model_receiver
llama-3.2-3b    mistral-0.2-7b    0.876678
                mistral-0.3-7b    0.759973
                qwen-2.5-7b       0.716260
mistral-0.2-7b  llama-3.2-3b      0.647476
                mistral-0.3-7b    0.781524
                qwen-2.5-1.5b     0.879528
                qwen-2.5-7b       0.791073
mistral-0.3-7b  llama-3.2-3b      0.630769
                mistral-0.2-7b    0.925000
                qwen-2.5-7b       0.753571
qwen-2.5-1.5b   llama-3.2-3b      0.640834
                mistral-0.2-7b    0.909596
                mistral-0.3-7b    0.784730
                qwen-2.5-7b       0.792185
qwen-2.5-7b     llama-3.2-3b      0.609322
                mistral-0.2-7b    0.935067
                mistral-0.3-7b    0.812459
                qwen-2.5-1.5b     0.886250
Name: abs, dtype: float64


/var/folders/p4/p76vy2ms4ygbm1tdzy5s9c_w0000gn/T/ipykernel_34271/2685527830.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['abs'] = df['delta_sarc'].abs()


In [463]:
check(results[results['round1_sarc_ratio'] >= 0.5])

model_sender    model_receiver
llama-3.2-3b    mistral-0.2-7b    0.700806
                mistral-0.3-7b    0.223937
                qwen-2.5-7b       0.173079
mistral-0.2-7b  llama-3.2-3b      0.151690
                mistral-0.3-7b    0.178286
                qwen-2.5-1.5b     0.879528
                qwen-2.5-7b       0.227193
mistral-0.3-7b  llama-3.2-3b      0.213253
                mistral-0.2-7b    0.820000
                qwen-2.5-7b       0.353608
qwen-2.5-1.5b   llama-3.2-3b      0.048191
                mistral-0.2-7b    0.370714
                mistral-0.3-7b    0.061278
                qwen-2.5-7b       0.077710
qwen-2.5-7b     llama-3.2-3b      0.107806
                mistral-0.2-7b    0.694136
                mistral-0.3-7b    0.191417
                qwen-2.5-1.5b     0.886250
Name: abs, dtype: float64


In [ ]:

results[results['flip_direction'] == 'down']

In [452]:
df_table = make_delta_table(results)

In [453]:
df_table

model_receiver,llama-3.2-3b,mistral-0.2-7b,mistral-0.3-7b,qwen-2.5-1.5b,qwen-2.5-7b,avg
model_sender,,,,,,
llama-3.2-3b_A_mean,NaN,0.8900,0.2500,0.9600,0.700,0.7000
llama-3.2-3b_ci95,NaN,0.0030,0.0120,0.0010,0.010,NaN
mistral-0.2-7b_A_mean,0.210,NaN,0.1800,0.9700,0.350,0.4275
mistral-0.2-7b_ci95,0.005,NaN,0.0040,0.0000,0.007,NaN
mistral-0.3-7b_A_mean,0.890,0.9200,NaN,0.9700,0.950,0.9325
mistral-0.3-7b_ci95,0.007,0.0030,NaN,0.0000,0.004,NaN
qwen-2.5-1.5b_A_mean,0.050,0.3700,0.0600,NaN,0.080,0.1400
qwen-2.5-1.5b_ci95,0.001,0.0030,0.0010,NaN,0.001,NaN
qwen-2.5-7b_A_mean,0.350,0.8700,0.2000,0.9700,NaN,0.5975


In [454]:
print(make_latex_body(df_table, float_fmt="{:.2f}"))

 & \texttt{llama-3.2-3b} & \texttt{mistral-0.2-7b} & \texttt{mistral-0.3-7b} & \texttt{qwen-2.5-1.5b} & \texttt{qwen-2.5-7b} & \texttt{Avg.} \\
\midrule
\texttt{llama-3.2-3b}&-- & 0.89 & 0.25 & 0.96 & 0.70 & 0.70 \\
 &  & \footnotesize{(±0.00)} & \footnotesize{(±0.01)} & \footnotesize{(±0.00)} & \footnotesize{(±0.01)} &  \\
\texttt{mistral-0.2-7b}&0.21 & -- & 0.18 & 0.97 & 0.35 & 0.43 \\
 & \footnotesize{(±0.01)} &  & \footnotesize{(±0.00)} & \footnotesize{(±0.00)} & \footnotesize{(±0.01)} &  \\
\texttt{mistral-0.3-7b}&0.89 & 0.92 & -- & 0.97 & 0.95 & 0.93 \\
 & \footnotesize{(±0.01)} & \footnotesize{(±0.00)} &  & \footnotesize{(±0.00)} & \footnotesize{(±0.00)} &  \\
\texttt{qwen-2.5-1.5b}&0.05 & 0.37 & 0.06 & -- & 0.08 & 0.14 \\
 & \footnotesize{(±0.00)} & \footnotesize{(±0.00)} & \footnotesize{(±0.00)} &  & \footnotesize{(±0.00)} &  \\
\texttt{qwen-2.5-7b}&0.35 & 0.87 & 0.20 & 0.97 & -- & 0.60 \\
 & \footnotesize{(±0.01)} & \footnotesize{(±0.01)} & \footnotesize{(±0.01)} & \footnotes

### The same tabel, just split into up and down flips.

In [466]:
up_df = make_delta_table(results, filter='up')
up_df

model_receiver,llama-3.2-3b,mistral-0.2-7b,mistral-0.3-7b,qwen-2.5-1.5b,qwen-2.5-7b,avg
model_sender,,,,,,
llama-3.2-3b_A_mean,NaN,0.9100,0.890000,0.9600,0.9300,0.9225
llama-3.2-3b_ci95,NaN,0.0030,0.023000,0.0010,0.0040,NaN
mistral-0.2-7b_A_mean,0.910,NaN,0.980000,0.9700,0.9700,0.9575
mistral-0.2-7b_ci95,0.006,NaN,0.039000,0.0000,0.0040,NaN
mistral-0.3-7b_A_mean,0.910,0.9200,NaN,0.9700,0.9600,0.9400
mistral-0.3-7b_ci95,0.004,0.0030,NaN,0.0000,0.0030,NaN
qwen-2.5-1.5b_A_mean,NaN,0.8400,NaN,NaN,0.9300,0.8850
qwen-2.5-1.5b_ci95,NaN,0.0230,NaN,NaN,0.0280,NaN
qwen-2.5-7b_A_mean,0.910,0.9000,0.910000,0.9700,NaN,0.9225


In [ ]:
down_df = make_delta_table(results, filter='down')
down_df

model_receiver,llama-3.2-3b,mistral-0.2-7b,mistral-0.3-7b,qwen-2.5-1.5b,qwen-2.5-7b,avg
model_sender,,,,,,
llama-3.2-3b_A_mean,NaN,0.700,0.2200,NaN,0.1700,0.363333
llama-3.2-3b_ci95,NaN,0.024,0.0120,NaN,0.0110,NaN
mistral-0.2-7b_A_mean,0.150,NaN,0.1800,0.880,0.2300,0.360000
mistral-0.2-7b_ci95,0.004,NaN,0.0040,0.019,0.0060,NaN
mistral-0.3-7b_A_mean,0.210,0.820,NaN,NaN,0.3500,0.460000
mistral-0.3-7b_ci95,0.050,0.218,NaN,NaN,0.0600,NaN
qwen-2.5-1.5b_A_mean,0.050,0.370,0.0600,NaN,0.0800,0.140000
qwen-2.5-1.5b_ci95,0.001,0.003,0.0010,NaN,0.0010,NaN
qwen-2.5-7b_A_mean,0.110,0.690,0.1900,0.890,NaN,0.470000


In [468]:
print(make_latex_body(up_df, float_fmt="{:.2f}"))

 & \texttt{llama-3.2-3b} & \texttt{mistral-0.2-7b} & \texttt{mistral-0.3-7b} & \texttt{qwen-2.5-1.5b} & \texttt{qwen-2.5-7b} & \texttt{Avg.} \\
\midrule
\texttt{llama-3.2-3b}&-- & 0.91 & 0.89 & 0.96 & 0.93 & 0.92 \\
 &  & \footnotesize{(±0.00)} & \footnotesize{(±0.02)} & \footnotesize{(±0.00)} & \footnotesize{(±0.00)} &  \\
\texttt{mistral-0.2-7b}&0.91 & -- & 0.98 & 0.97 & 0.97 & 0.96 \\
 & \footnotesize{(±0.01)} &  & \footnotesize{(±0.04)} & \footnotesize{(±0.00)} & \footnotesize{(±0.00)} &  \\
\texttt{mistral-0.3-7b}&0.91 & 0.92 & -- & 0.97 & 0.96 & 0.94 \\
 & \footnotesize{(±0.00)} & \footnotesize{(±0.00)} &  & \footnotesize{(±0.00)} & \footnotesize{(±0.00)} &  \\
\texttt{qwen-2.5-1.5b}&-- & 0.84 & -- & -- & 0.93 & 0.89 \\
 &  & \footnotesize{(±0.02)} &  &  & \footnotesize{(±0.03)} &  \\
\texttt{qwen-2.5-7b}&0.91 & 0.90 & 0.91 & 0.97 & -- & 0.92 \\
 & \footnotesize{(±0.01)} & \footnotesize{(±0.00)} & \footnotesize{(±0.02)} & \footnotesize{(±0.00)} &  &  \\
\midrule
\texttt{avg}&0.91

In [469]:
print(make_latex_body(down_df, float_fmt="{:.2f}"))

 & \texttt{llama-3.2-3b} & \texttt{mistral-0.2-7b} & \texttt{mistral-0.3-7b} & \texttt{qwen-2.5-1.5b} & \texttt{qwen-2.5-7b} & \texttt{Avg.} \\
\midrule
\texttt{llama-3.2-3b}&-- & 0.70 & 0.22 & -- & 0.17 & 0.36 \\
 &  & \footnotesize{(±0.02)} & \footnotesize{(±0.01)} &  & \footnotesize{(±0.01)} &  \\
\texttt{mistral-0.2-7b}&0.15 & -- & 0.18 & 0.88 & 0.23 & 0.36 \\
 & \footnotesize{(±0.00)} &  & \footnotesize{(±0.00)} & \footnotesize{(±0.02)} & \footnotesize{(±0.01)} &  \\
\texttt{mistral-0.3-7b}&0.21 & 0.82 & -- & -- & 0.35 & 0.46 \\
 & \footnotesize{(±0.05)} & \footnotesize{(±0.22)} &  &  & \footnotesize{(±0.06)} &  \\
\texttt{qwen-2.5-1.5b}&0.05 & 0.37 & 0.06 & -- & 0.08 & 0.14 \\
 & \footnotesize{(±0.00)} & \footnotesize{(±0.00)} & \footnotesize{(±0.00)} &  & \footnotesize{(±0.00)} &  \\
\texttt{qwen-2.5-7b}&0.11 & 0.69 & 0.19 & 0.89 & -- & 0.47 \\
 & \footnotesize{(±0.01)} & \footnotesize{(±0.02)} & \footnotesize{(±0.01)} & \footnotesize{(±0.02)} &  &  \\
\midrule
\texttt{avg}&0.13

## Majority label distribution: Round 1 

In [ ]:
def majority_label_table(df):
    df = df.copy()
    df['label'] = df['sarc_ratio'].apply(
        lambda x: 'sarcastic' if x >= 0.5 else 'literal'
    )
    df['label'] = df['label'].astype(
        pd.CategoricalDtype(categories=['sarcastic', 'literal'])
    )
    counts = df.groupby(['model', 'label'], observed=False).size().reset_index(name='count')

    counts['value'] = ((counts['count'] / counts.groupby('model')['count'].transform('sum')) * 100)

    counts_sarc = counts[counts['label'] == 'sarcastic'][['model', 'value']]
    counts_sarc.set_index('model', inplace = True)
    counts_sarc.rename(columns = {'value': 'Sarcastic %'}, inplace = True)
    return counts_sarc

In [430]:
df = pd.read_csv('results/first-results-sarc-ratio.csv')

In [431]:
df_majority_label = majority_label_table(df)
print(df_majority_label.to_latex(float_format="%.2f"))

\begin{tabular}{lr}
\toprule
 & Sarcastic % \\
model &  \\
\midrule
llama-3.2-1b & 100.00 \\
llama-3.2-3b & 96.85 \\
mistral-0.2-7b & 89.17 \\
mistral-0.3-7b & 99.63 \\
qwen-2.5-1.5b & 28.11 \\
qwen-2.5-7b & 95.30 \\
\bottomrule
\end{tabular}



# Plots 

I need to create the following dataset 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# optional: enforce a consistent order of models
model_order = sorted(heatmap_df.index.unique())
heatmap_df = heatmap_df.reindex(index=model_order, columns=model_order)

sns.set(style="white")  # clean background

fig, ax = plt.subplots(figsize=(6, 5))

hm = sns.heatmap(
    heatmap_df,
    ax=ax,
    cmap="YlOrRd",      # yellow → red
    annot=True,         # write values in the cells
    fmt="d",            # integer formatting; use ".2f" for floats
    linewidths=0.5,
    linecolor="white",
    square=True,        # make cells square
    cbar_kws={"shrink": 0.8}
)

# axis labels
ax.set_xlabel("Model 2")
ax.set_ylabel("Model 1")

# put x tick labels on top
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position("top")

# rotate x tick labels
plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
plt.setp(ax.get_yticklabels(), rotation=0)

plt.tight_layout()
plt.show()
